In [2]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import random
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
wandb.login(key='44fa96b263794ea519fb29399eb6b8f469eb934b')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
##44fa96b263794ea519fb29399eb6b8f469eb934b

In [5]:
train_dataset = pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_train.csv',header=None)
test_dataset=pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_test.csv',header=None)
valid_dataset=pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_valid.csv' ,header=None)


In [6]:
lang2_ = 'HINDI'
SOW_token = 0 #<
EOW_token = 1 #>
UNK_token = 2 #_
PAD_token = 3 #@
#fix_this 



In [7]:
class make_dict:
    def __init__(self,name):
        self.name=name
        self.chartoindex={}
        self.indextochar={0: "<", 1: ">",2:"_",3:'@'}
        self.n_chars=4
    
    def addchar(self,word):
        for char in word:
            if char not in self.chartoindex:
                self.chartoindex[char]=self.n_chars
                self.indextochar[self.n_chars]=char
                self.n_chars+=1
            

In [8]:
# doing it on train a dataframe is given as input having input output 
# and it make the disctionary based on that and return input_dict_obj,outout_dict_obj
#and the pair's as input ,output
def Input_Output_train(df):
    #convert the dataframe to list 
    input_word=df.iloc[:,0].tolist()
    output_word=df.iloc[:,1].tolist()
    input_lang=make_dict('ENG')
    output_lang=make_dict('HINDI')
    
    # to make a dictionary of characters for the input language 
    for word in input_word:
        input_lang.addchar(word)
   # to make a dictionary of characters for teh output language 
    for word in output_word:
        output_lang.addchar(word)
    return input_lang,output_lang,input_word,output_word
  

In [9]:
#use to get max length of the input dataframe provided 
def getlength_x(df):
    input_list = df.iloc[:,0].tolist()
    input_list_length=[]
    for word in input_list:
        input_list_length.append(len(word))
    
    input_max_length=max(input_list_length)
    return input_max_length+1


In [10]:
#to get max length of the output 
def getlength_y(df):
    output_list =df.iloc[:,1].tolist()
    output_list_length=[]
    for word in output_list:
        output_list_length.append(len(word))
    output_max_length=max(output_list_length)
    return output_max_length+1
    
    

In [11]:
input_word_dict,output_word_dict,inputword_list,outputword_list=Input_Output_train(train_dataset)


ENG 30
HINDI 68


In [12]:
def getlistofwords_x(df):
    return df.iloc[:,0].tolist()

def getlistofwords_y(df):
    return df.iloc[:,1].tolist()

In [13]:
valid_input_list = getlistofwords_x(valid_dataset)

In [14]:
#to form tensor list pass the wordlist,maxlength,dictusing 
def tensorsFormation(inputword,max_length,input_dict):
    #convering the input and output to tensor
    
    inlist=[]
    for char in inputword:
        inlist.append(input_dict.chartoindex[char])
    inlist.append(EOW_token)
    diff=max_length-len(inlist)
    inlist.extend([PAD_token]*diff)
    wordtensor=torch.tensor(inlist, dtype=torch.long,device=device)
        
    
    return wordtensor

In [17]:
def batchcreation(data,batchsize):
    size=len(data)
    #print(size)
    batchdata=[]
    for i in range(0,size,batchsize):
        temp=torch.stack(data[i:i+batchsize]).to(device)
        batchdata.append(temp.transpose(0,1))
        #batchdata.append((nn.utils.rnn.pad_sequence(data[i:i+batch_size])).to(device))
    
    return batchdata

In [19]:
class EncoderRNN(nn.Module):
    def __init__(self, parameters):
        super(EncoderRNN, self).__init__()
        self.input_size=parameters['input_size_encoder']
        self.dropout = nn.Dropout(parameters['dropout_encoder'])
        self.dropout_val=parameters['dropout_encoder']
        self.num_layers = parameters['num_layers']
        self.batch_size = parameters['batch_size']
        self.embedding_size = parameters['embedding_size']
        self.hidden_size = parameters['hidden_size']
        self.cell_type = parameters['cell_type']
        self.embedding = nn.Embedding(parameters['input_size_encoder'], parameters['embedding_size'])
        self.bidirection = parameters['bidirectional']
        
        if(self.cell_type == "GRU" ):
            self.cell_calc = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val, bidirectional = self.bidirection)
        elif(self.cell_type == "LSTM"):
            self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val, bidirectional = self.bidirection)
        elif(self.cell_type == "RNN"):
            self.cell_calc = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val, bidirectional = self.bidirection)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(-1,self.batch_size, self.embedding_size)
        output = self.dropout(embedded)
        if(self.cell_type == "LSTM"):
            _,(hidden, cell) = self.lstm(output)
        else:
            _,hidden = self.cell_calc(output, hidden)
                    
        if (self.bidirection==True):
            #rehsaping the hidden layer to calculate the avg of the two layers 
            
            dim1=hidden.size(1)
            dim2=hidden.size(2)
            hidden = hidden.reshape(2, -1, dim1, dim2)
            #print(hidden.shape)
            temp = hidden[0]
            temp.add(hidden[1])
            hidden=0.5*temp
            
            
            if(self.cell_type == "LSTM"):
                dim1= cell.size(1)
                dim2= cell.size(2)
                cell = cell.reshape(2,-1,dim1,dim2)
                temp=cell[0]
                temp.add(cell[1])
                cell=0.5*temp
        if self.cell_type == "LSTM":
            return hidden, cell
        else:
            return hidden

    def initHidden(self):
        if self.bidirection:
            return torch.zeros(2*self.num_layers, self.batch_size, self.hidden_size).to(device)
        else:
            return torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device)

In [20]:
class DecoderRNN(nn.Module):
    def __init__(self, parameters):
        super(DecoderRNN, self).__init__()
        self.output_size=parameters['output_size_decoder']
        self.dropout = nn.Dropout(parameters['dropout_decoder'])
        self.dropout_val=parameters['dropout_decoder']
        self.num_layers = parameters['num_layers']
        self.batch_size = parameters['batch_size']
        self.embedding_size = parameters['embedding_size']
        self.hidden_size = parameters['hidden_size']
        self.cell_type = parameters['cell_type']
        self.embedding = nn.Embedding(parameters['output_size_decoder'], parameters['embedding_size'])
               
       
        if(self.cell_type == "GRU"):
            self.cell_calc = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val)
        elif(self.cell_type == "RNN"):
            self.cell_calc = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val)
        elif(self.cell_type == "LSTM"):
            self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val)
        
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(-1, self.batch_size, self.embedding_size)
        output = self.dropout(output)
        output = torch.relu(output)
        if(self.cell_type == "LSTM"):
            output, (hidden, cell) = self.lstm(output, (hidden[0], hidden[1]))
        else:
            output, hidden = self.cell_calc(output, hidden)

        if self.cell_type == "LSTM":
            return self.softmax(self.out(output[0])),hidden,cell
        return self.softmax(self.out(output[0])),hidden

    def initHidden(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size, device=device)

In [21]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,train_iter_params):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    loss = 0
    if train_iter_params['cell_type'] == "LSTM":
        encoder_hidden, encoder_cell = encoder(input_tensor, encoder_hidden)
    else:
        encoder_hidden = encoder(input_tensor, encoder_hidden)
        
    decoder_input = torch.tensor([SOW_token]*train_iter_params['batch_size']).to(device)
    
#     print(encoder_output.shape, encoder_hidden.shape)

    decoder_hidden = encoder_hidden
    if train_iter_params['cell_type'] == "LSTM":
        decoder_cell = encoder_cell

    if(random.random() < teacher_forcing_ratio):
        use_teacher_forcing = True 
    else:
        use_teacher_forcing=False
        
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if train_iter_params['cell_type'] == "LSTM":
                decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, (decoder_hidden, decoder_cell))
            else:
                #print(decoder_input.shape,decoder_hidden.shape)
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            if train_iter_params['cell_type'] == "LSTM":
                decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, (decoder_hidden, decoder_cell))
            else:
                #print(decoder_input.shape,decoder_hidden.shape)
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [34]:
def train_setup(encoder, decoder, train_iter_params,parameters):
    
    
    Total_loss = 0  # Reset every print_every
    run_name = "EBS_{}_NL_{}_HL_{}_CellT_{}_BS_{}".format(parameters['embedding_size'], parameters['num_layers'], parameters['hidden_size'], parameters['cell_type'], parameters['batch_size'])
    
    encoder_optimizer = optim.NAdam(encoder.parameters(), lr=train_iter_params['learning_rate'], weight_decay = 0.0005)
    decoder_optimizer = optim.NAdam(decoder.parameters(), lr=train_iter_params['learning_rate'], weight_decay = 0.0005)
    criterion = nn.CrossEntropyLoss()
    
    input_tensor_list=[]

    for i in range(len(inputword_list)):
        if(i%train_iter_params['batch_size']==0):
            max_length_x=getlength_x(train_dataset[i:i+train_iter_params['batch_size']])

        inputwordtensor =tensorsFormation(inputword_list[i],max_length_x,input_word_dict)
        input_tensor_list.append(inputwordtensor)
    
    output_tensor_list=[]

    for i in range(len(outputword_list)):
        if(i%train_iter_params['batch_size']==0):
            max_length_y=getlength_y(train_dataset[i:i+train_iter_params['batch_size']])
        outputwordtesor = tensorsFormation(outputword_list[i],max_length_y,output_word_dict)
        output_tensor_list.append(outputwordtesor)

    valid_iptensor_list=[]

    for i in range(len(valid_input_list)):
        if(i%train_iter_params['batch_size']==0):
            valid_max_length_x=getlength_x(valid_dataset[i:i+train_iter_params['batch_size']])
        validipwordtensor =tensorsFormation(valid_input_list[i],valid_max_length_x,input_word_dict)
        valid_iptensor_list.append(validipwordtensor)

    
    
 #   input_tensor_list=creatensorlist_x(train_dataset,inputword_list,parameters['batch_size'],input_word_dict)
#     output_tensor_list=creatensorlist_y(train_dataset,outputword_list,train_iter_params['batch_size'],output_word_dict)
#     valid_iptensor_list=creatensorlist_x(valid_dataset,valid_input_list,train_iter_params['batch_size'],input_word_dict)
    
    
    train_batch_input = batchcreation(input_tensor_list, train_iter_params['batch_size'])
    train_batch_target = batchcreation(output_tensor_list, train_iter_params['batch_size'])
    valid_batch_input = batchcreation(valid_iptensor_list, train_iter_params['batch_size'])
    no_of_batches=len(train_batch_target)
    no_of_datapoints=train_iter_params['batch_size']*no_of_batches
   
        
    for count in range(train_iter_params['epochs']):
        #print("Epoch ", count ," started ", datetime.now())
        print('epoch count is ',count+1 ,'time' ,datetime.now())
        for i in range(no_of_batches):
            loss = train(train_batch_input[i], train_batch_target[i], encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion,train_iter_params)
            Total_loss += loss*train_iter_params['batch_size']
        
        

        avg_loss = Total_loss/no_of_datapoints
        Total_loss = 0
        print("Average loss after ", count+1, "epochs is ", avg_loss)
#        train_accuracy = cal_Accuracy(train_batch_input, train_output, cell_type, len(train_input), batch_size)
#         print("Train accuracy is ", train_accuracy)

        valid_output_list = valid_dataset.iloc[:,1]
        valid_accuracy = cal_Accuracy(valid_batch_input, valid_output_list, train_iter_params['cell_type'], train_iter_params['batch_size'],encoder,decoder)
        print("Valid accuracy is ", valid_accuracy)
        wandb.log({"validation_accuracy": valid_accuracy, "training_loss": avg_loss, 'epoch': count})
    wandb.run.name = run_name
    wandb.run.save()
    wandb.run.finish()


In [23]:
def evaluate(encoder, decoder, input_tensors, cell_type,batch_size):
    with torch.no_grad():
        
        input_length = input_tensors.size(0)
        predicted_word_tensor=[]
        encoder_hidden = encoder.initHidden()

        if cell_type == "LSTM":
            encoder_hidden, encoder_cell = encoder(input_tensors, encoder_hidden)
        else:
            encoder_hidden = encoder(input_tensors, encoder_hidden)

        decoder_input = torch.tensor([SOW_token]*batch_size, device=device)  # SOW

        decoder_hidden = encoder_hidden
        if cell_type == "LSTM":
            decoder_cell = encoder_cell
            
        for di in range(input_length):
            
            if cell_type == "LSTM":
                decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, (decoder_hidden, decoder_cell))
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
                                    
            temp=topi.view(1,-1).squeeze()
            predicted_word_tensor.append(temp)
            decoder_input = topi.squeeze().detach()
            
        predicted_word_tensor=torch.stack(predicted_word_tensor,dim=1).to(device)
        
        return predicted_word_tensor
        

In [24]:
def cal_Accuracy(input, actual_output, cell_type, batch_size,encoder1,decoder1):
    correct_output=0
    no_of_batch=len(input)
    no_of_datapoints=batch_size*no_of_batch 
    incorrect_correct_pair=[]
    
    for i in range(no_of_batch):
        output_tensor_predicted = evaluate(encoder1, decoder1, input[i], cell_type,batch_size)
                
        for j in range(output_tensor_predicted.size(0)):
            string=""
            for k in range(output_tensor_predicted.size(1)):
                target_val=output_tensor_predicted[j][k].item()
                if(target_val==EOW_token or target_val==3):
                    break
                else:
                    string =string+output_word_dict.indextochar[target_val]
            
            #print(string)
            if(actual_output[i*batch_size+j]==string):
                correct_output+=1
           # else:
           #     incorrect_correct_pair.append([])
                
                
              
    #print('no of correct output =',correct_output/n *100)
    
    return correct_output/no_of_datapoints *100
            
  
    

cuda


In [27]:
# paramet= {

# "input_size_encoder":input_word_dict.n_chars,
# "output_size_decoder":output_word_dict.n_chars,
# "hidden_size":  512,
# "num_layers":  2,
# "dropout_encoder":0.4,
# "dropout_decoder": 0.3,
# "batch_size": 32,
# "embedding_size": 256,
# "epochs": 10,
# "cell_type":"RNN",
# "bidirectional": True
    
# }

In [28]:
# encodertest = EncoderRNN(paramet).to(device)
# decodertest = DecoderRNN(paramet).to(device)

# train_iter_params={
#     'epochs':paramet['epochs'],
#     'learning_rate':0.001,
#     'batch_size':paramet['batch_size'],
#     'cell_type':paramet['cell_type']  
    
    
#  }
# train_setup(encodertest, decodertest, train_iter_params,paramet)
    

In [29]:
def runSweep():
    
    config_defaults = {
        "embedding_size": 64,
        "num_layers": 3,
        "hidden_layer": 256,
        "cell_type": "LSTM",
        "bidirectional": True,
        "dropout_encoder": 0.2,
        "dropout_decoder": 0.3,
        "epochs": 20,
        "batch_size": 32,
    }
    wandb.init( entity = 'singhbhavesh999', project = 'DLSeqtoseq',config=config_defaults)
    
    param= {

        "input_size_encoder":input_word_dict.n_chars,
        "output_size_decoder":output_word_dict.n_chars,
        "hidden_size":  wandb.config.hidden_layer,
        "num_layers":  wandb.config.num_layers,
        "dropout_encoder":wandb.config.dropout_encoder,
        "dropout_decoder": wandb.config.dropout_decoder,
        "batch_size": wandb.config.batch_size,
        "embedding_size": wandb.config.embedding_size,
        "epochs": wandb.config.epochs,
        "cell_type":wandb.config.cell_type,
        "bidirectional": wandb.config.bidirectional

        }
    


    
    encoder1 = EncoderRNN(param).to(device)
    decoder1 = DecoderRNN(param).to(device)
    train_iter_params={
        'epochs':param['epochs'],
        'learning_rate':0.001,
        'batch_size':param['batch_size'],
        'cell_type':param['cell_type']  
    }
    
 
    train_setup(encoder1, decoder1, train_iter_params,param)

In [35]:
sweep_config = {
  
  "metric": {
      "name":"validation_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        "embedding_size": {
            "values": [512, 256, 64, 32]
        },
        "num_layers": {
            "values": [3, 2, 1]
        },
        "hidden_layer": {
            "values": [512, 256, 128]
        },
        "cell_type": {
            "values": ["RNN","GRU","LSTM"]
        },
        "bidirectional": {
            "values": [True,False]
        },
        "dropout_encoder": {
            "values": [0.2, 0.3, 0.4]
        },
        "dropout_decoder": {
            "values": [0.2, 0.3, 0.4]
        },
        "epochs": {
            "values": [10,15]
        },
        "batch_size": {
            "values": [256, 128, 64, 32]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, entity="singhbhavesh999", project="DLSeqtoseq")
wandb.agent(sweep_id, runSweep, count = 100)

Create sweep with ID: xl00ecq9
Sweep URL: https://wandb.ai/singhbhavesh999/DLSeqtoseq/sweeps/xl00ecq9


wandb: Agent Starting Run: ad4cdmho with config:
wandb: 	batch_size: 256
wandb: 	bidirectional: True
wandb: 	cell_type: GRU
wandb: 	dropout_decoder: 0.2
wandb: 	dropout_encoder: 0.4
wandb: 	embedding_size: 512
wandb: 	epochs: 10
wandb: 	hidden_layer: 512
wandb: 	num_layers: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.
wandb: Ctrl + C detected. Stopping sweep.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 1144, in init
    run = wi.init()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 801, in init
    run_start_result = run_start_handle.wait(timeout=30)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/mailbox.py", line 283, in wait
    found, abandoned = self._slot._get_and_clear(timeout=wait_timeout)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/mailbox.py", line 130, 

{'embedding_size': {'values': [512, 256, 64, 32]},
 'num_layers': {'values': [3, 2, 1]},
 'hidden_layer': {'values': [512, 256, 128]},
 'cell_type': {'values': ['RNN']},
 'bidirectional': {'values': [True]},
 'dropout_encoder': {'values': [0.2, 0.3, 0.4]},
 'dropout_decoder': {'values': [0.2, 0.3, 0.4]},
 'epochs': {'values': [10]},
 'batch_size': {'values': [256, 128, 64, 32]}}

In [32]:
# wandb.agent('a0km7f3z', runSweep, count = 2)

NameError: name 'batch_size' is not defined

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 1144, in init
    run = wi.init()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 801, in init
    run_start_result = run_start_handle.wait(timeout=30)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/mailbox.py", line 283, in wait
    found, abandoned = self._slot._get_and_clear(timeout=wait_timeout)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/mailbox.py", line 130, in _get_and_clear
    if self._wait(timeout=timeout):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/mailbox.py", line 126, in _wait
    return self._event.wait(timeout=timeout)
  File "/opt/conda/lib/python3.7/threading.py", line 552, in wait
    signaled = self._cond.wait(timeout)
  File "/opt/conda/lib/python3.7/threading.py", line 300, in wait
    gotit = waiter.acquire(True, timeout)
Exception

During handling of the above exception, a